In [20]:
import numpy as np
from scipy.signal import savgol_filter
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
from sklearn.metrics import classification_report, confusion_matrix
from pathlib import Path
import json

In [21]:
def compute_derivatives(x, y):
    vx = np.gradient(x)
    vy = np.gradient(y)
    ax = np.gradient(vx)
    ay = np.gradient(vy)
    speed = np.sqrt(vx**2 + vy**2)
    accel = np.sqrt(ax**2 + ay**2)
    return vx, vy, ax, ay, speed, accel


def sliding_window_stats(arr, t, w):
    left = max(0, t - w)
    right = min(len(arr), t + w + 1)
    window = arr[left:right]
    return window.mean(), window.std()


def build_features(ball_data, windows=(3, 5, 9)):
    """
    Build per-frame features (NO LABEL).
    Returns:
        X: np.array [n_frames, n_features]
        frame_ids: list of frame indices
    """
    frame_ids = sorted(ball_data.keys(), key=lambda x: int(x))
    x = np.array([ball_data[f]["x"] for f in frame_ids], dtype=float)
    y = np.array([ball_data[f]["y"] for f in frame_ids], dtype=float)
    visible = np.array([ball_data[f]["visible"] for f in frame_ids])

    # Interpolation simple
    idx = np.arange(len(x))
    mask = visible & ~np.isnan(x)
    x[~mask] = np.interp(idx[~mask], idx[mask], x[mask])
    y[~mask] = np.interp(idx[~mask], idx[mask], y[mask])

    # Smoothing
    if len(x) >= 7:
        x = savgol_filter(x, 7, 3)
        y = savgol_filter(y, 7, 3)

    vx, vy, ax, ay, speed, accel = compute_derivatives(x, y)

    X = []
    valid_frames = []

    for t in range(3, len(x) - 3):
        feats = [
            y[t],
            vx[t], vy[t],
            ax[t], ay[t],
            speed[t],
            accel[t],
            vy[t] - vy[t-1],
            speed[t+1] - speed[t],
        ]

        for w in windows:
            feats.extend(sliding_window_stats(speed, t, w))
            feats.extend(sliding_window_stats(accel, t, w))
            feats.extend(sliding_window_stats(vy, t, w))

        X.append(feats)
        valid_frames.append(t)

    return np.array(X), valid_frames, frame_ids

In [22]:
def detect_events_unsupervised(X, contamination=0.03):
    """
    Unsupervised anomaly detection.
    Returns indices of anomalous frames (relative to X).
    """
    model = IsolationForest(
        n_estimators=200,
        contamination=contamination,
        random_state=42,
        n_jobs=-1,
    )
    model.fit(X)
    preds = model.predict(X)   # -1 = anomaly
    return np.where(preds == -1)[0]


def cluster_events(frame_indices, eps=5):
    """
    Temporal clustering of anomalies.
    """
    if len(frame_indices) == 0:
        return []

    X = frame_indices.reshape(-1, 1)
    labels = DBSCAN(eps=eps, min_samples=1).fit_predict(X)

    centers = []
    for lab in np.unique(labels):
        cluster = frame_indices[labels == lab]
        centers.append(int(np.median(cluster)))

    return sorted(centers)

In [23]:
def classify_events(event_frames, y, vy, speed, accel):
    """
    Physics-based hit vs bounce.
    """
    bounce = []
    hit = []

    accel_thr = np.percentile(accel, 80)
    y_ground = np.percentile(y, 70)

    for t in event_frames:
        if t < 2 or t > len(y) - 3:
            continue

        # Bounce conditions
        is_bounce = (
            vy[t-1] > 0 and vy[t+1] < 0 and
            y[t] >= y_ground and
            accel[t] > accel_thr
        )

        if is_bounce:
            bounce.append(t)
        else:
            hit.append(t)

    return bounce, hit

In [24]:
def unsupervised_hit_bounce_detection(ball_data):
    X, valid_frames, frame_ids = build_features(ball_data)

    anomaly_idx = detect_events_unsupervised(X)
    anomaly_frames = np.array(valid_frames)[anomaly_idx]

    event_centers = cluster_events(anomaly_frames)

    # Recompute trajectories for physics
    x = np.array([ball_data[f]["x"] for f in frame_ids], dtype=float)
    y = np.array([ball_data[f]["y"] for f in frame_ids], dtype=float)
    vy = np.gradient(y)
    speed = np.sqrt(np.gradient(x)**2 + np.gradient(y)**2)
    accel = np.gradient(speed)

    bounce, hit = classify_events(event_centers, y, vy, speed, accel)

    bounce_set = set(bounce)
    hit_set = set(hit)

    for i, fid in enumerate(frame_ids):
        if i in bounce_set:
            ball_data[fid]["pred_action"] = "bounce"
        elif i in hit_set:
            ball_data[fid]["pred_action"] = "hit"
        else:
            ball_data[fid]["pred_action"] = "air"

    return ball_data

In [25]:
def evaluate(ball_data):
    y_true = []
    y_pred = []

    for f in sorted(ball_data.keys(), key=lambda x: int(x)):
        y_true.append(ball_data[f]["action"])
        y_pred.append(ball_data[f]["pred_action"])

    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))

In [26]:
input_dir = Path(
    "Data hit & bounce/per_point_v2"
)

y_true_all = []
y_pred_all = []

n_points = 0
n_frames = 0

for json_path in input_dir.glob("*.json"):
    with open(json_path, "r") as f:
        ball_data = json.load(f)

    ball_data = unsupervised_hit_bounce_detection(ball_data)

    for k in ball_data.keys():
        y_true_all.append(ball_data[k]["action"])
        y_pred_all.append(ball_data[k]["pred_action"])
        n_frames += 1

    n_points += 1

print("===================================")
print("GLOBAL MATCH EVALUATION")
print("===================================")
print(f"Number of points  : {n_points}")
print(f"Number of frames  : {n_frames}")
print()

print("Classification report (global):")
print(
    classification_report(
        y_true_all,
        y_pred_all,
        labels=["air", "hit", "bounce"],
        digits=3,
        zero_division=0,
    )
)

print("Confusion matrix (rows=true, cols=pred):")
print(
    confusion_matrix(
        y_true_all,
        y_pred_all,
        labels=["air", "hit", "bounce"]
    )
)

GLOBAL MATCH EVALUATION
Number of points  : 313
Number of frames  : 177341

Classification report (global):
              precision    recall  f1-score   support

         air      0.984     0.996     0.990    174295
         hit      0.292     0.163     0.209      1600
      bounce      0.000     0.000     0.000      1446

    accuracy                          0.981    177341
   macro avg      0.425     0.387     0.400    177341
weighted avg      0.970     0.981     0.975    177341

Confusion matrix (rows=true, cols=pred):
[[173679    616      0]
 [  1339    261      0]
 [  1428     18      0]]


The unsupervised physics-based method correctly identifies candidate event regions, but fails at exact frame-level classification due to extreme class imbalance and the impulse-like nature of hits and bounces

This motivated the supervised sliding-window approach, which learns temporal context instead of relying on local derivatives.